In [39]:
import requests
import matplotlib.pyplot as plt
import numpy as np
import lxml.html
from lxml import etree
import xml.etree.ElementTree as ET
import string
import csv
import numpy as np
import time

def AbsolutePaths(tags):
    tmp=''
    for tag in tags:
        tmp+=tag.tag+'/'
    return str(tmp)    

def xpath_generator(C):
  X = []
  for link in C:
    #ritorna la get
    link = ''.join(link)
    link = link[:len(link)-1]
    response = requests.get(link)
    #ritorna l'elemento html della pagina
    tree = lxml.html.fromstring(response.text)
    #ritorna la lista dei nodi foglia
    childs = tree.xpath('//*[not(child::*)]')
    #funzione che calcola il path assoluto radice-foglia come stringa
    #calcolo della lista che contiene gli xpath assoluti alle foglie della pagina 
    for child in childs:
      #ciascun elemento in X è una quadrupla (implementata come una lista di 4 elementi) formata da:
      #url della pagina, xpath di ciascun nodo della pagina, contenuto del nodo, peso del nodo
      xpath_element = []
      tags = child.xpath('ancestor::node()')
      path = AbsolutePaths(tags)
      path += str(child.tag)
      contenuto = child.text_content()
      xpath_element.append(link)
      xpath_element.append(path)
      xpath_element.append(contenuto)
      xpath_element.append(0)
      X.append(xpath_element)
  return X

#Ricerca la frequenza e la frequenza del contenuto di un certo xpath
#e la cardinalità dell'insieme di tutti i contenuti associati a quell'xpath
def find_f(url, xpath, contenuto, lista):
  frequenza = 1
  f_contenuto = 1
  #t_i è l'insieme di tutti i contenuti associati a xpath
  t_i = [contenuto]
  #lista dei url già visti nella ricerca
  url_visti = []
  url_visti.append(url)
  for new_path in lista:
    #se allo stesso xpath ho un contenuto diverso da quelli che già ho lo inserisco in t_i
    if new_path[1] == xpath and new_path[2] not in t_i:
      t_i.append(new_path[2])
    #calcolo della frequenza e di f_contenuto
    if new_path[0] not in url_visti:
      if new_path[1] == xpath:
        url_visti.append(new_path[0])
        frequenza += 1
        if new_path[2] == contenuto:
          f_contenuto += 1
  return frequenza, f_contenuto, len(t_i)

#lista = [(url,xpath,contenuto,peso), (....), (....), .... ]
def calcolo_peso_xpath(lista, num_pagine):
  for xpath in lista:
    #prendo xpath
    x = xpath[1]
    url = xpath[0]
    contenuto = xpath[2]
    #calcolo la frequenza, la frequenza del contenuto e la cardinalità di t_i
    frequenza, f_contenuto, t_i = find_f(url, x, contenuto, lista)
    #calcolo della informatività
    info = 1 - (f_contenuto / (num_pagine * t_i))
    #calcolo del peso del xpath
    peso = frequenza * info
    #inserisco il peso nella lista
    xpath[3] = peso
    
def somma_pesi_per_pagina(X):
  url_pesi = {}
  for e in X:
    k = e[0]
    if k not in url_pesi.keys():
      url_pesi[k] = e[3]
    else:
      url_pesi[k] += e[3]
  return url_pesi

#Trova la pagina (ritornando il suo url) con la somma dei pesi maggiore
def find_p(X, M):
  calcolo_peso_xpath(X, M)
  pagine_con_pesi = somma_pesi_per_pagina(X)
  peso_max = 0
  pagina_max = ''
  for k in pagine_con_pesi.keys():
    peso_pagina = pagine_con_pesi[k]
    if peso_pagina > peso_max:
      pagina_max = k
      peso_max = peso_pagina
  return pagina_max, peso_max

#Crea un dizionario con chiave xpath e come valore il contenuto
def build_dict(X,p):
  d = {}
  for e in X:
    if e[0] == p:
      if e[1] not in d:
        xpath = e[1]
        content = e[2]
        d[xpath] = content
  return d

#Rimuove la pagina da X e gli Xpath associati ad essa
def remove_p(X, p):
  X1 = []
  for e in X:
    if e[0] != p:
      X1.append(e)
  #elimino gli xpath contenuto in p nelle altre pagine
  X2 = []
  dict_xpath_content = build_dict(X,p)
  for e in X1:
    if e[1] not in dict_xpath_content:
      X2.append(e)
    elif e[2] != dict_xpath_content[e[1]]:
        X2.append(e)  
  return X2

#Main
def page_annotation(C, K):
  X = xpath_generator(C)
  print('XPath totali: ' + str(len(X)))
  S = []
  pesi = []
  while X!=[] and len(S)<=K:
    p , peso = find_p(X, len(C))
    print(p)
    S.append(p)
    pesi.append(peso)
    X = remove_p(X, p)
  return S, X, pesi

# Carico il set di dati che voglio analizzare

In [21]:
site = []
with open('rotten.csv', 'r') as file:
    for line in file:
        site.append(line)

In [22]:
len(site)

209446

In [23]:
del site[0]

In [30]:
i=0
train=[]
test=[]
for url in site:
    if i<(len(site)/2):
        train.append(url)
    else:
        test.append(url)
    i+=1

print(len(train))
print(len(test))

104723
104722


In [49]:
train50 = train[:50]
test50 = test[:50]
train_50_2 = train[52:102]
print(train_50_2[0])

https://www.rottentomatoes.com/m/madagascar

https://www.rottentomatoes.com/m/longest_yard



In [40]:
s1, x1, p1 = page_annotation(train50,5)

print(len(s1))
print('XPath rimanenti: ' + str(len(x1)))

XPath totali: 30271
https://www.rottentomatoes.com/m/pacifier
https://www.rottentomatoes.com/m/fever_pitch
https://www.rottentomatoes.com/m/star_wars_episode_iii_revenge_of_the_sith
https://www.rottentomatoes.com/m/robots
https://www.rottentomatoes.com/m/lot_like_love
https://www.rottentomatoes.com/m/interpreter
6
XPath rimanenti: 9867


In [50]:
s2, x2, p2 = page_annotation(train_50_2,5)

print(len(s2))
print('XPath rimanenti: ' + str(len(x2)))

XPath totali: 28673
https://www.rottentomatoes.com/m/fantastic_four
https://www.rottentomatoes.com/m/mr_and_mrs_smith
https://www.rottentomatoes.com/m/island
https://www.rottentomatoes.com/m/rebound
https://www.rottentomatoes.com/m/lords_of_dogtown
https://www.rottentomatoes.com/m/bad_news_bears
6
XPath rimanenti: 9267


In [52]:
primo1 = p1[0]
primo2 = p2[0]
somma1 = 0
somma2 = 0
for p in p1:
    somma1 += p
for p in p2:
    somma2 += p

somma1 = somma1 - primo1
somma2 = somma2 - primo2

precision1 = primo1/somma1
precision2 = primo2/somma2
recall = primo2/primo1

print('Precision train = ' + str(precision1))
print('Precision test = ' + str(precision2))
print('Recall = ' + str(recall))

Precision train = 0.4599334842506192
Precision test = 0.48855216251045225
Recall = 0.9779650340413484
